# **Análisis de Datos Públicos de Viajeros**

El propósito de este tutorial es presentarle un workflow general para la descarga y procesamiento de datos públicos del BioPortal. Se eligió el API de viajeros ya que es uno de los APIs más sencillos y livianos para manejar. De tal manera, se espera que ayude a aliviar el learning curve a quienes no hayan programado usando Python o Pandas antes.

# Setup

Lo único que se hace en esta sección es importar los módulos requeridos para analizar los datos.<br>Entre ellos:


*   **bioportal** - Descarga los datos disponibles en los APIs públicos
*   **pandas** - Manejo de tablas, datos, estadisticas, etc. Increiblemente util y el más que se le dara uso aqui.
*   **plotly** - Gráficas interactivas. Facilitan visualizar los datos descargados y procesados.





In [1]:
#@title Importar modulos
%%capture
# Instalar modulo de Bioportal y darle update al modulo de graficar (plotly)
!pip install --upgrade plotly
!pip install bioportal

# Importar modulos requeridos
import datetime
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.colab import data_table
from bioportal import BioPortalClient
from google.colab import data_table

In [2]:
# Fecha de hoy (usado para circumventar datos que aparecen con fechas en el futuro)
date_today = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-4))).strftime('%Y-%m-%d')
date_today

'2021-05-07'

# Travel Declaration Forms (TDFs)

El TDF es el formulario que cada viajero completa al llegar a Puerto Rico. Estos documentos recopilan información como: si el viajero es un residente o no-residente, si trajo resultado negativo de prueba, etc.

Para poder descargar los datos, se abre un "cliente" de los APIs públicos de BioPortal. Esto facilita extraer los datos sin sobrecomplicarse con como hacer las conexiones apropiadas.
<br>

**Nota**: No todos los APIs públicos listados aqui funcionan con el cliente pero puede intentar ni modo. Los de pruebas toman al menos 5-10 minutos en descargar.

In [3]:
#@title Cliente de APIs públicos de BioPortal
cliente = BioPortalClient()
cliente.datasets_disponibles()

Bases de datos disponibles:
  1   'Cantidades totales de pruebas reportadas'
  2   'Pruebas unicas con informacion minima'
  3   'Pruebas unicas con ID de paciente y fechas en tiempo local de Puerto Rico'
  4   'Pruebas unicas con ID de paciente y fechas en tiempo internacional UTC'
  5   'Pruebas diarias para grafica de dashboard de Salud'
  6   'Pruebas por fecha de coleccion'
  7   'Pruebas por fecha de reporte'
  8   'Pruebas por fecha de coleccion y entidad'
  9   'Total de TDF por fecha reportada de llegada'
 10   'Total de TDF por municipio'
 11   'Casos por fecha de coleccion'
 12   'Casos por fecha de creacion en sistema'
 13   'Casos por grupo de edad'
 14   'Casos por ciudad'
 15   'Casos por region'
 16   'Resumen de Escuelas Publicas y Privadas'


## Descarga de datos de viajeros

Los datos son colocados en un formato de tabla, y se reorganizan un poco para que su uso sea mas conveniente:


1.   Descarga de datos a una tabla
2.   Convertir las fechas de llegada a formato de fecha
3.   Fijar las fechas como la referencia (el índice) para seleccionar filas
4.   Limitar la tabla a fechas de llegada tomando lugar despues del 1 de Julio de 2020



In [4]:
# Datos publicos de Travel Declaration Forms (TDFs)
print("Esto puede que tome unos minutos...")

tdf_fecha = cliente.descargar_dataset('Total de TDF por fecha reportada de llegada') # 1.
tdf_fecha['date'] = tdf_fecha['date'].apply(pd.to_datetime) # 2.
tdf_fecha.set_index('date', inplace=True) # 3.
tdf_fecha = tdf_fecha.loc['2020-07-01':] # 4.

tdf_fecha

Esto puede que tome unos minutos...
Descargando "Total de TDF por fecha reportada de llegada"...
Descargado.


,total,notScanned,scanned,percentageTravelersScanned,arrivedWithoutResults,arrivedWithResults,percentageArrivedWithNegativePcrResults,arrivedWithTestPerformedAtPortOfEntry,arrivedWithNegativeTestPerformedAtPortOfEntry,residents,residentsArrivedWithResults,percentageResidentsArrivedWithNegativePcrResults,nonResidents,nonResidentsArrivedWithResults,nonResidentsStayingLessThan5Days,percentageNonResidentsStayingLessThan5DaysArrivedWithNegativePcrResults,nonResidentsStaying5DaysOrMore,percentageResidentsStaying5DaysOrMoreArrivedWithNegativePcrResults
date,,,,,,,,,,,,,,,,,,
2020-07-13,10,0,10,100.0,5,5,20.000000,0,0,2,0,0.000000,8,5,0,0.000000,8,0.000000
2020-07-14,40,0,40,100.0,31,9,12.500000,0,0,11,3,9.090909,29,6,0,0.000000,29,0.000000
2020-07-15,1465,0,1465,100.0,1298,167,11.058020,9,9,699,31,4.005722,766,136,0,0.000000,766,100.000000
2020-07-16,4678,0,4678,100.0,3823,855,18.170158,25,25,1480,88,5.878378,3198,767,1024,19.042969,2174,25.966448
2020-07-17,5134,0,5134,100.0,4076,1058,20.529801,25,23,1805,184,10.193906,3329,874,1182,28.172589,2147,30.300860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-30,1,1,0,0.0,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000
2022-04-29,3,3,0,0.0,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000
2022-05-01,1,1,0,0.0,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000


Debajo y a la izquierda de la tabla, aparece un texto que dice "N rows x M columns", donde N corresponde al numero de filas y M al numero de columnas. N en este ejemplo particular representa el total de fechas consideradas, y M el total de variables disponibles para cada fecha.<br>

<br>

Para obtener un listado especifico de cada uno, podemos accesar los "index" y los "columns" para las filas y las columnas, respectivamente:


In [5]:
#@title Filas / Fechas de la tabla
tdf_fecha.index

DatetimeIndex(['2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16',
               '2020-07-17', '2020-07-18', '2020-07-19', '2020-07-20',
               '2020-07-21', '2020-07-22',
               ...
               '2021-05-28', '2021-05-29', '2021-05-30', '2021-06-01',
               '2021-06-05', '2021-07-30', '2022-04-29', '2022-05-01',
               '2022-05-29', '2023-04-23'],
              dtype='datetime64[ns]', name='date', length=318, freq=None)

In [6]:
#@title Columnas / Variables de la tabla
tdf_fecha.columns.tolist()

['total',
 'notScanned',
 'scanned',
 'percentageTravelersScanned',
 'arrivedWithoutResults',
 'arrivedWithResults',
 'percentageArrivedWithNegativePcrResults',
 'arrivedWithTestPerformedAtPortOfEntry',
 'arrivedWithNegativeTestPerformedAtPortOfEntry',
 'residents',
 'residentsArrivedWithResults',
 'percentageResidentsArrivedWithNegativePcrResults',
 'nonResidents',
 'nonResidentsArrivedWithResults',
 'nonResidentsStayingLessThan5Days',
 'percentageNonResidentsStayingLessThan5DaysArrivedWithNegativePcrResults',
 'nonResidentsStaying5DaysOrMore',
 'percentageResidentsStaying5DaysOrMoreArrivedWithNegativePcrResults']

## Accesando información

Aqui brevemente cubriremos como extraer información particular de la tabla. Para información más detallada, puede buscar en google sobre como hacer "slices" a DataFrames (las tablas) usando el modulo de Pandas.

Para accesar información particular de la tabla, se usará mayormente el método de "localización", abreviado como '.loc' :



```python
tabla.loc['nombre de fila', 'nombre de columna']
```



### Días específicos

Para elegir días especificos, hay que "localizar" ese día dentro de la tabla:

```python
tabla.loc['fecha']
```
Esto se permite porque, al descargar los datos, establecimos la columna de fechas como el "index".

En este ejemplo localizamos los datos correspondientes al 13 de Marzo de 2021:

In [7]:
tdf_fecha.loc['2021-03-13']

total                                                                      14792.000000
notScanned                                                                   932.000000
scanned                                                                    13860.000000
percentageTravelersScanned                                                    93.699297
arrivedWithoutResults                                                       5575.000000
arrivedWithResults                                                          8285.000000
percentageArrivedWithNegativePcrResults                                       55.935641
arrivedWithTestPerformedAtPortOfEntry                                          0.000000
arrivedWithNegativeTestPerformedAtPortOfEntry                                  0.000000
residents                                                                   3598.000000
residentsArrivedWithResults                                                  848.000000
percentageResidentsArrivedWithNe

Se incluyen todas las columnas / variables ya que no especificamos alguna en particular. Para elegir solo una variable, se le añade al localizador:

In [8]:
tdf_fecha.loc['2021-03-13', 'total']

14792.0

### Variables específicas

Idéntico a como se accesan días específicos, excepto que no se escribe el '.loc' :


```python
tabla['nombre de columna']
```



Por ejemplo, para extraer el total de viajeros llegando diariamente, se extrae la columna del 'total':

In [9]:
tdf_fecha['total']

date
2020-07-13      10
2020-07-14      40
2020-07-15    1465
2020-07-16    4678
2020-07-17    5134
              ... 
2021-07-30       1
2022-04-29       3
2022-05-01       1
2022-05-29       1
2023-04-23       1
Name: total, Length: 318, dtype: int64

### Subconjuntos de días

Tambien se puede elegir una colección específica de días. Para esto, se añade los dos puntos ':' al localizador:

```python
tabla.loc['nombre de primera fila' : 'nombre de última fila']
```

De esta manera, puede elegir colecciones como los viajeros que llegaron dentro del mes de Marzo:

In [10]:
tdf_fecha.loc['2021-03-01':'2021-03-31']

,total,notScanned,scanned,percentageTravelersScanned,arrivedWithoutResults,arrivedWithResults,percentageArrivedWithNegativePcrResults,arrivedWithTestPerformedAtPortOfEntry,arrivedWithNegativeTestPerformedAtPortOfEntry,residents,residentsArrivedWithResults,percentageResidentsArrivedWithNegativePcrResults,nonResidents,nonResidentsArrivedWithResults,nonResidentsStayingLessThan5Days,percentageNonResidentsStayingLessThan5DaysArrivedWithNegativePcrResults,nonResidentsStaying5DaysOrMore,percentageResidentsStaying5DaysOrMoreArrivedWithNegativePcrResults
date,,,,,,,,,,,,,,,,,,
2021-03-01,10054,520,9534,94.827929,6121,3413,33.887010,1,1,5883,954,16.882426,3895,2459,1097,71.194166,2798,66.732155
2021-03-02,8201,510,7691,93.781246,4795,2896,35.239605,1,0,4129,661,17.037229,3823,2235,994,68.410463,2829,60.576293
2021-03-03,9572,670,8902,93.000418,4543,4359,45.434601,0,0,3522,735,22.585622,5661,3624,1945,69.562982,3716,66.029623
2021-03-04,11819,867,10952,92.664354,4101,6851,57.872916,0,0,2690,645,26.039564,8475,6206,4220,77.819905,4255,71.526508
2021-03-05,12139,964,11175,92.058654,4380,6795,55.820084,0,0,2973,834,30.740741,8475,5961,3925,72.662420,4550,73.621206
2021-03-06,11305,807,10498,92.861566,4737,5761,50.880142,0,0,3664,856,25.583483,7156,4905,1948,71.201232,5208,73.857404
2021-03-07,11479,879,10600,92.342539,6121,4479,38.966809,0,0,5621,953,18.209640,5372,3526,1316,72.796353,4056,72.059430
2021-03-08,10682,792,9890,92.585658,6108,3782,35.330462,1,0,5777,913,17.155205,4568,2869,1436,71.169916,3132,66.136631
2021-03-09,9595,701,8894,92.694112,5378,3516,36.602397,1,1,4332,703,17.496267,4876,2813,1326,65.007541,3550,62.086011


Si solo necesitas establecer una fecha de comienzo o una fecha de final, puedes omitir la fecha que no necesitas.

Para todos los viajeros que han entrado desde el comienzo del 2021 hasta ahora:

In [11]:
tdf_fecha.loc['2021-01-01':]

,total,notScanned,scanned,percentageTravelersScanned,arrivedWithoutResults,arrivedWithResults,percentageArrivedWithNegativePcrResults,arrivedWithTestPerformedAtPortOfEntry,arrivedWithNegativeTestPerformedAtPortOfEntry,residents,residentsArrivedWithResults,percentageResidentsArrivedWithNegativePcrResults,nonResidents,nonResidentsArrivedWithResults,nonResidentsStayingLessThan5Days,percentageNonResidentsStayingLessThan5DaysArrivedWithNegativePcrResults,nonResidentsStaying5DaysOrMore,percentageResidentsStaying5DaysOrMoreArrivedWithNegativePcrResults
date,,,,,,,,,,,,,,,,,,
2021-01-01,8086,506,7580,93.742271,5761,1819,22.495672,0,0,4416,382,9.182692,3420,1437,1027,47.127556,2393,44.926414
2021-01-02,11227,726,10501,93.533446,9078,1423,12.612452,4,4,8629,519,6.306527,2319,904,467,46.038544,1852,47.222222
2021-01-03,12335,686,11649,94.438589,10541,1108,8.974463,2,2,10469,525,5.288607,1722,583,330,39.393939,1392,41.644205
2021-01-04,12659,777,11882,93.862074,10675,1207,9.518919,2,2,10496,446,4.501315,1996,761,458,45.851528,1538,44.596443
2021-01-05,10847,738,10109,93.196275,8732,1377,12.657878,3,3,8431,525,6.607707,2194,852,443,45.372460,1751,44.551282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-30,1,1,0,0.000000,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000
2022-04-29,3,3,0,0.000000,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000
2022-05-01,1,1,0,0.000000,0,0,0.000000,0,0,0,0,0.000000,0,0,0,0.000000,0,0.000000


Para todos los viajeros que entraron hasta el final del 2020:

In [12]:
tdf_fecha.loc[:'2020-12-31']

,total,notScanned,scanned,percentageTravelersScanned,arrivedWithoutResults,arrivedWithResults,percentageArrivedWithNegativePcrResults,arrivedWithTestPerformedAtPortOfEntry,arrivedWithNegativeTestPerformedAtPortOfEntry,residents,residentsArrivedWithResults,percentageResidentsArrivedWithNegativePcrResults,nonResidents,nonResidentsArrivedWithResults,nonResidentsStayingLessThan5Days,percentageNonResidentsStayingLessThan5DaysArrivedWithNegativePcrResults,nonResidentsStaying5DaysOrMore,percentageResidentsStaying5DaysOrMoreArrivedWithNegativePcrResults
date,,,,,,,,,,,,,,,,,,
2020-07-13,10,0,10,100.000000,5,5,20.000000,0,0,2,0,0.000000,8,5,0,0.000000,8,0.000000
2020-07-14,40,0,40,100.000000,31,9,12.500000,0,0,11,3,9.090909,29,6,0,0.000000,29,0.000000
2020-07-15,1465,0,1465,100.000000,1298,167,11.058020,9,9,699,31,4.005722,766,136,0,0.000000,766,100.000000
2020-07-16,4678,0,4678,100.000000,3823,855,18.170158,25,25,1480,88,5.878378,3198,767,1024,19.042969,2174,25.966448
2020-07-17,5134,0,5134,100.000000,4076,1058,20.529801,25,23,1805,184,10.193906,3329,874,1182,28.172589,2147,30.300860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,10809,808,10001,92.524748,7939,2062,19.021186,1,1,6432,408,6.721011,3990,1654,660,48.636364,3330,47.653257
2020-12-28,10706,749,9957,93.003923,8063,1894,17.653652,3,3,6563,393,6.322768,3773,1501,702,49.145299,3071,43.880926
2020-12-29,10626,718,9908,93.242989,7518,2390,22.473179,2,2,5572,474,9.057902,4675,1916,912,47.697368,3763,43.676603


### Subconjuntos de variables

Aqui se provee al localizador una lista especifica de variables para extraer en vez de una sola variable:

```python
tabla[ ['nombre de 1era columna', 'nombre de 2da columna', 'etc'] ]
```

Para recopilar el total de viajeros recibido, y su desglose en cuanto a residencia, se seleccionan esas 3 columnas particulares:

In [13]:
tdf_fecha[ ['total', 'residents', 'nonResidents'] ]

,total,residents,nonResidents
date,,,
2020-07-13,10,2,8
2020-07-14,40,11,29
2020-07-15,1465,699,766
2020-07-16,4678,1480,3198
2020-07-17,5134,1805,3329
...,...,...,...
2021-07-30,1,0,0
2022-04-29,3,0,0
2022-05-01,1,0,0


### Combinaciones de ambas

El localizador puede recibir combinaciones de los inputs ya explicados para devolver resultados más especificos.

In [14]:
tdf_fecha.loc[ '2021-03-01':'2021-03-31', ['total', 'residents', 'nonResidents'] ]

,total,residents,nonResidents
date,,,
2021-03-01,10054,5883,3895
2021-03-02,8201,4129,3823
2021-03-03,9572,3522,5661
2021-03-04,11819,2690,8475
2021-03-05,12139,2973,8475
2021-03-06,11305,3664,7156
2021-03-07,11479,5621,5372
2021-03-08,10682,5777,4568
2021-03-09,9595,4332,4876


## Manipulando variables

Al tener los datos organizados por filas y columnas, estos se pueden combinar de esa misma manera.

Aqui se resumirá como procesarlos para rellenar información que ya esta implícita en la tabla

### Total de viajeros llegando con resultado negativo

La tabla ya llega con una columna listando el total de viajeros (residentes + no-residentes) llegando diariamente, pero para la cantidad de estos llegando con PCR negativa solo incluye un porciento.<br>

Estas variables se pueden multiplicar para encontrar la frecuencia de estos viajeros que si llegan con PCR- y de vez asignarla devuelta a la tabla

In [15]:
tdf_fecha['arrivedWithNegativePcrResults'] = (tdf_fecha['total'] * tdf_fecha['percentageArrivedWithNegativePcrResults'] / 100).astype(int)
tdf_fecha['arrivedWithNegativePcrResults']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


date
2020-07-13       2
2020-07-14       5
2020-07-15     161
2020-07-16     850
2020-07-17    1054
              ... 
2021-07-30       0
2022-04-29       0
2022-05-01       0
2022-05-29       0
2023-04-23       0
Name: arrivedWithNegativePcrResults, Length: 318, dtype: int64

### Residentes llegando con resultado negativo

El mismo cálculo puede ser realizado pero para el número diario de residentes llegando diariamente

In [16]:
tdf_fecha['residentsArrivedWithNegativePcrResults'] = (tdf_fecha['residents'] * tdf_fecha['percentageResidentsArrivedWithNegativePcrResults'] / 100).astype(int)
tdf_fecha['residentsArrivedWithNegativePcrResults']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


date
2020-07-13      0
2020-07-14      1
2020-07-15     28
2020-07-16     87
2020-07-17    184
             ... 
2021-07-30      0
2022-04-29      0
2022-05-01      0
2022-05-29      0
2023-04-23      0
Name: residentsArrivedWithNegativePcrResults, Length: 318, dtype: int64

### No-residentes llegando con resultado negativo

El cálculo que falta puede ser realizado al combinar los dos anteriores:

In [17]:
tdf_fecha['nonResidentsArrivedWithNegativePcrResults'] = (tdf_fecha['arrivedWithNegativePcrResults'] - tdf_fecha['residentsArrivedWithNegativePcrResults']).astype(int)
tdf_fecha['nonResidentsArrivedWithNegativePcrResults']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


date
2020-07-13      2
2020-07-14      4
2020-07-15    133
2020-07-16    763
2020-07-17    870
             ... 
2021-07-30      0
2022-04-29      0
2022-05-01      0
2022-05-29      0
2023-04-23      0
Name: nonResidentsArrivedWithNegativePcrResults, Length: 318, dtype: int64

### Resultado

Y asi se pueden combinar columnas para crean nuevas variables que sean mas faciles de analizar y/o presentar.

In [18]:
tdf_fecha[['arrivedWithNegativePcrResults', 'residentsArrivedWithNegativePcrResults', 'nonResidentsArrivedWithNegativePcrResults']]

,arrivedWithNegativePcrResults,residentsArrivedWithNegativePcrResults,nonResidentsArrivedWithNegativePcrResults
date,,,
2020-07-13,2,0,2
2020-07-14,5,1,4
2020-07-15,161,28,133
2020-07-16,850,87,763
2020-07-17,1054,184,870
...,...,...,...
2021-07-30,0,0,0
2022-04-29,0,0,0
2022-05-01,0,0,0


## Graficando variables

Esta sección se enfoca en incluir ejemplos de como presentar los datos en estas tablas en gráficos interactivos. No es un tutorial para Plotly, el modulo de graficar, pero mas una galería para demostrar ejemplos.

In [19]:
#@title
# Ventana_Dias = 7 #@param {type:"slider", min:1, max:14, step:1}

roll_days = 7 # Tamaño de ventana de promedio, en días

# Crear el gráfico como tal
fig = px.line(tdf_fecha, # Tabla
              x=tdf_fecha.index, # Valores de x (usualmente el índice)
              y=tdf_fecha['total'].rolling(roll_days).mean(), # Valores de Y. Promedio móvil
              # y=tdf_fecha['arrivedWithNegativePcrResults'].rolling(roll_days).mean(), # Valores de Y. Promedio móvil
              )


# Para añadir mas lineas ("traces")
# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['total'].rolling(7).sum()))


# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['arrivedWithResults'].rolling(7).sum()))


# Poner bonito los ejes de X y Y, con titulo y en el rango correcto
fig.update_xaxes(title='<b>Fecha de Llegada', range=['2020-10-01',date_today])
fig.update_yaxes(title='<b>TDFs', range=[0, 15e3])


# Cambiar el layout de la grafica
fig.update_layout(template='plotly_white', # Pone un theme sencillo de colores
                  title='<b>Viajeros Llegando<br>(Promedio de {} días)'.format(roll_days), # Título del gráfico
                  title_x=0.5, # Centralizar el título (50% horizontal)
                  width=800, # Achicar el ancho del gráfico (mas facil enviar imagen por WhatsApp)
                  )

fig.show() # Enseñar la gráfica interactiva

In [20]:

pd.DataFrame(data=tdf_fecha['arrivedWithNegativePcrResults'].values,
             index=tdf_fecha.index.strftime('%B %Y').fillna('Unknown').rename('Month'),
            #  columns=['total']
             columns=['arrivedWithNegativePcrResults']
             ).groupby('Month').sum()\
.loc[['July 2020','August 2020','September 2020','October 2020','November 2020','December 2020', 'January 2021', 'February 2021', 'March 2021', 'April 2021']]
# .to_csv('TDF_Mensual.csv')

,arrivedWithNegativePcrResults
Month,
July 2020,11848
August 2020,24697
September 2020,28113
October 2020,48776
November 2020,49887
December 2020,65003
January 2021,65529
February 2021,109373
March 2021,190871


In [21]:
#@title
# Ventana_Dias = 7 #@param {type:"slider", min:1, max:14, step:1}

roll_days = 1 # Tamaño de ventana de promedio, en días

# Crear el gráfico como tal
fig = px.line(tdf_fecha, # Tabla
              x=tdf_fecha.index, # Valores de x (usualmente el índice)
              y=(tdf_fecha['total']-tdf_fecha['arrivedWithNegativePcrResults']).rolling(roll_days).mean(), # Valores de Y. Promedio móvil
              # y=tdf_fecha['arrivedWithNegativePcrResults'].rolling(roll_days).mean(), # Valores de Y. Promedio móvil
              )


# Para añadir mas lineas ("traces")
# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['total'].rolling(7).sum()))


# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['arrivedWithResults'].rolling(7).sum()))


# Poner bonito los ejes de X y Y, con titulo y en el rango correcto
fig.update_xaxes(title='<b>Fecha de Llegada', range=['2020-10-01',date_today])
fig.update_yaxes(title='<b>TDFs', range=[0, 15e3])


# Cambiar el layout de la grafica
fig.update_layout(template='plotly_white', # Pone un theme sencillo de colores
                  title='<b>Viajeros Llegando sin prueba negativa<br>(Promedio de {} días)'.format(roll_days), # Título del gráfico
                  title_x=0.5, # Centralizar el título (50% horizontal)
                  width=800, # Achicar el ancho del gráfico (mas facil enviar imagen por WhatsApp)
                  )

fig.show() # Enseñar la gráfica interactiva

In [22]:
#@title
fig = go.Figure()

# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['arrivedWithNegativePcrResults'].rolling(7).mean() / tdf_fecha['total'].rolling(7).mean(), name='Total'))

fig.add_trace(go.Scatter(x=tdf_fecha.index,
                         y=tdf_fecha['residents'].rolling(7).mean() / tdf_fecha['total'].rolling(7).mean(), name='Residentes' ))

# fig.add_trace(go.Scatter(x=tdf_fecha.index,
#                          y=tdf_fecha['nonResidents'].rolling(7).mean() / tdf_fecha['total'].rolling(7).mean(), name='No Residentes' ))

fig.update_xaxes(title='<b>Fecha de Llegada', range=['2020-09-27',date_today])
fig.update_yaxes(title='<b>Promedio de TDFs diarios', tickformat='%', range=[0,1])

fig.update_layout(template='plotly_white', title='<b>Porción de Residentes fuera de Viajeros Llegando', title_x=0.5)
fig.show()

In [23]:
#@title
# fig = px.line(x=tdf_fecha.index, y=tdf_fecha['arrivedWithNegativePcrResults'].rolling(7).mean() / tdf_fecha['total'].rolling(7).mean(), label='total')
fig = go.Figure()

fig.add_trace(go.Scatter(x=tdf_fecha.index,
                         y=tdf_fecha['arrivedWithNegativePcrResults'].rolling(7).mean() / tdf_fecha['total'].rolling(7).mean(), name='Total'))

fig.add_trace(go.Scatter(x=tdf_fecha.index,
                         y=tdf_fecha['residentsArrivedWithNegativePcrResults'].rolling(7).mean() / tdf_fecha['residents'].rolling(7).mean(), name='Residentes' ))

fig.add_trace(go.Scatter(x=tdf_fecha.index,
                         y=tdf_fecha['nonResidentsArrivedWithNegativePcrResults'].rolling(7).mean() / tdf_fecha['nonResidents'].rolling(7).mean(), name='No Residentes' ))

fig.update_xaxes(title='<b>Fecha de Llegada', range=['2020-09-27',date_today])
fig.update_yaxes(title='<b>Porciento de promedio<br>de TDFs diarios con prueba', tickformat='%', range=[0,1])

fig.update_layout(template='plotly_white', title='<b>Viajeros Llegando con Pruebas Negativas', title_x=0.5)
fig.show()

In [24]:
tdf_fecha['residentsArrivedWithNegativePcrResults'].loc['2021-01-01':].sum() / tdf_fecha['residents'].loc['2021-01-01':].sum()

0.1933169733062905

In [25]:
#@title
fig = go.Figure()

roll_days = 7
normed = False


fig.add_trace(go.Scatter(
    x=tdf_fecha.index,
    y=tdf_fecha['residents'].rolling(roll_days).mean(),
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(111, 231, 219)'),
    stackgroup='one', # define stack group
    groupnorm='percent' if normed else None,
    name='Residentes'
))

fig.add_trace(go.Scatter(
    x=tdf_fecha.index,
    y=tdf_fecha['nonResidents'].rolling(roll_days).mean(),
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one', # define stack group
    name='No Residentes'
))

fig.update_xaxes(range=['2020-10-01', date_today], title='<b>Fecha de llegada')

if normed:
  fig.update_yaxes(ticksuffix='%', range=[1, 100], title='<b>Fracción del total recibido')
else:
  fig.update_yaxes(title='Total recibido')

fig.update_layout(width=800, title='<b>Tipo de Viajeros (Promedio de {} días)</b>'.format(roll_days), title_x=0.22, template='plotly_white')

In [26]:
#@title
fig = go.Figure()

roll_days = 7
normed = False


fig.add_trace(go.Scatter(
    x=tdf_fecha.index,
    y=tdf_fecha['residentsArrivedWithNegativePcrResults'].rolling(roll_days).mean(),
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(111, 231, 219)'),
    stackgroup='one', # define stack group
    groupnorm='percent' if normed else None,
    name='Residentes'
))

fig.add_trace(go.Scatter(
    x=tdf_fecha.index,
    y=tdf_fecha['nonResidentsArrivedWithNegativePcrResults'].rolling(roll_days).mean(),
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one', # define stack group
    name='No Residentes'
))

fig.update_xaxes(range=['2020-10-01', date_today], title='<b>Fecha de llegada')

if normed:
  fig.update_yaxes(ticksuffix='%', range=[1, 100], title='<b>Fracción del Total Recibido')
else:
  fig.update_yaxes(title='Total Recibido')

fig.update_layout(width=800, title='<b>Tipo de Viajeros con PCR-<br>(Promedio de {} días)</b>'.format(roll_days), title_x=0.5, template='plotly_white')